# pypevol: Basic API Usage

This notebook demonstrates the core APIs and data models of pypevol, a package for analyzing Python package API evolution. You'll learn how to work with the fundamental building blocks: APIElement, VersionInfo, APIChange, and AnalysisResult objects.

First, let's import the pypevol models and other necessary libraries:

In [1]:
# Import pypevol models
from pypevol.models import (
    APIElement, APIType, VersionInfo, AnalysisResult, 
    APIChange, ChangeType
)

# Standard library imports
from datetime import datetime
import json
from pathlib import Path

print("✅ Successfully imported pypevol models!")

✅ Successfully imported pypevol models!


/home/ec2-user/miniconda3/envs/pymevol/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## 2. Create APIElement Objects

APIElement represents a single API component (function, class, method, etc.) in a Python package. Let's create different types of API elements:

In [2]:
# Create a function API element
function_api = APIElement(
    name="process_data",
    type=APIType.FUNCTION,
    module_path="mypackage.utils",
    signature="process_data(data: List[str], options: Dict = None) -> pd.DataFrame",
    docstring="Process raw data and return a pandas DataFrame",
    line_number=42,
    type_hints={"data": "List[str]", "options": "Dict", "return": "pd.DataFrame"}
)

# Create a class API element
class_api = APIElement(
    name="DataProcessor",
    type=APIType.CLASS,
    module_path="mypackage.core",
    docstring="Main class for processing data with various methods",
    line_number=15,
    metadata={"bases": ["BaseProcessor"], "is_exception": False}
)

# Create a method API element
method_api = APIElement(
    name="transform",
    type=APIType.METHOD,
    module_path="mypackage.core",
    signature="transform(self, data: Any) -> Any",
    docstring="Transform input data using configured rules",
    line_number=78,
    decorators=["@property", "@cached"]
)

# Create a constant API element
constant_api = APIElement(
    name="DEFAULT_CONFIG",
    type=APIType.CONSTANT,
    module_path="mypackage.config",
    line_number=5,
    metadata={"value": "{'timeout': 30, 'retries': 3}"}
)

print("Created API elements:")
for api in [function_api, class_api, method_api, constant_api]:
    print(f"  • {api.type.value}: {api.full_name}")
    if api.signature:
        print(f"    Signature: {api.signature}")
    print(f"    Private: {api.is_private}")

Created API elements:
  • function: mypackage.utils.process_data
    Signature: process_data(data: List[str], options: Dict = None) -> pd.DataFrame
    Private: False
  • class: mypackage.core.DataProcessor
    Private: False
  • method: mypackage.core.transform
    Signature: transform(self, data: Any) -> Any
    Private: False
  • constant: mypackage.config.DEFAULT_CONFIG
    Private: False


## 3. Working with APIType Enums

The APIType enum defines the different types of API elements that can be tracked:

In [3]:
# Explore APIType enum
print("Available API Types:")
for api_type in APIType:
    print(f"  {api_type.name}: {api_type.value}")

# Create APIs of each type
api_samples = {
    APIType.FUNCTION: APIElement("my_function", APIType.FUNCTION, "module"),
    APIType.CLASS: APIElement("MyClass", APIType.CLASS, "module"),
    APIType.METHOD: APIElement("my_method", APIType.METHOD, "module"),
    APIType.PROPERTY: APIElement("my_property", APIType.PROPERTY, "module"),
    APIType.CONSTANT: APIElement("MY_CONSTANT", APIType.CONSTANT, "module"),
    APIType.MODULE: APIElement("submodule", APIType.MODULE, "module")
}

print("\nAPI Type Examples:")
for api_type, api_element in api_samples.items():
    print(f"  {api_type.value}: {api_element.full_name}")

# Check if an element is of a specific type
print(f"\nIs function_api a FUNCTION? {function_api.type == APIType.FUNCTION}")
print(f"Is class_api a CLASS? {class_api.type == APIType.CLASS}")

Available API Types:
  FUNCTION: function
  CLASS: class
  METHOD: method
  PROPERTY: property
  CONSTANT: constant
  MODULE: module

API Type Examples:
  function: module.my_function
  class: module.MyClass
  method: module.my_method
  property: module.my_property
  constant: module.MY_CONSTANT
  module: module.submodule

Is function_api a FUNCTION? True
Is class_api a CLASS? True


## 4. Demonstrate Private API Detection

PyPevol automatically detects private APIs based on Python naming conventions:

In [4]:
# Create APIs with different privacy levels
public_api = APIElement("public_function", APIType.FUNCTION, "module")
private_api = APIElement("_private_function", APIType.FUNCTION, "module")
magic_api = APIElement("__magic_method__", APIType.METHOD, "module")
dunder_api = APIElement("__init__", APIType.METHOD, "module")

privacy_examples = [
    ("public_function", public_api),
    ("_private_function", private_api),
    ("__magic_method__", magic_api),
    ("__init__", dunder_api)
]

print("Privacy Detection Results:")
print("-" * 40)
for name, api in privacy_examples:
    privacy_status = "🔒 Private" if api.is_private else "🌐 Public"
    print(f"{name:<20} → {privacy_status}")

print("\nNaming Convention Rules:")
print("• Names starting with single underscore (_) → Private")
print("• Names starting with double underscore (__) → Public (magic methods)")
print("• Regular names → Public")

# You can override the automatic detection if needed
manual_private = APIElement("should_be_private", APIType.FUNCTION, "module")
manual_private.is_private = True  # Manually set as private
print(f"\nManually set private: {manual_private.name} → {'🔒 Private' if manual_private.is_private else '🌐 Public'}")

Privacy Detection Results:
----------------------------------------
public_function      → 🌐 Public
_private_function    → 🔒 Private
__magic_method__     → 🌐 Public
__init__             → 🌐 Public

Naming Convention Rules:
• Names starting with single underscore (_) → Private
• Names starting with double underscore (__) → Public (magic methods)
• Regular names → Public

Manually set private: should_be_private → 🔒 Private


## 5. Create VersionInfo Objects

VersionInfo represents metadata about a specific version of a package:

In [5]:
# Create version info for different releases
version_1_0 = VersionInfo(
    version="1.0.0",
    release_date=datetime(2023, 1, 15),
    python_requires=">=3.8",
    dependencies=["requests>=2.25.0", "pandas>=1.3.0"],
    wheel_url="https://pypi.org/project/mypackage/1.0.0/mypackage-1.0.0-py3-none-any.whl",
    metadata={
        "summary": "Initial release of MyPackage",
        "author": "Package Author",
        "license": "MIT"
    }
)

version_1_1 = VersionInfo(
    version="1.1.0",
    release_date=datetime(2023, 3, 20),
    python_requires=">=3.8",
    dependencies=["requests>=2.25.0", "pandas>=1.3.0", "click>=8.0.0"],
    wheel_url="https://pypi.org/project/mypackage/1.1.0/mypackage-1.1.0-py3-none-any.whl",
    metadata={
        "summary": "Added CLI support and bug fixes",
        "author": "Package Author",
        "license": "MIT"
    }
)

version_2_0 = VersionInfo(
    version="2.0.0",
    release_date=datetime(2023, 6, 10),
    python_requires=">=3.9",  # Dropped Python 3.8 support
    dependencies=["requests>=2.28.0", "pandas>=1.5.0", "click>=8.0.0"],
    wheel_url="https://pypi.org/project/mypackage/2.0.0/mypackage-2.0.0-py3-none-any.whl",
    metadata={
        "summary": "Major version with breaking changes",
        "author": "Package Author",
        "license": "MIT"
    }
)

versions = [version_1_0, version_1_1, version_2_0]

print("Package Version History:")
print("=" * 50)
for version in versions:
    print(f"Version: {version.version}")
    print(f"  Released: {version.release_date.strftime('%Y-%m-%d')}")
    print(f"  Python: {version.python_requires}")
    print(f"  Dependencies: {len(version.dependencies)}")
    print(f"  Summary: {version.metadata.get('summary', 'N/A')}")
    print()

Package Version History:
Version: 1.0.0
  Released: 2023-01-15
  Python: >=3.8
  Dependencies: 2
  Summary: Initial release of MyPackage

Version: 1.1.0
  Released: 2023-03-20
  Python: >=3.8
  Dependencies: 3
  Summary: Added CLI support and bug fixes

Version: 2.0.0
  Released: 2023-06-10
  Python: >=3.9
  Dependencies: 3
  Summary: Major version with breaking changes



## 6. Create APIChange Objects

APIChange represents a modification to an API element between versions:

In [6]:
# Create different types of API changes
added_change = APIChange(
    element=function_api,
    change_type=ChangeType.ADDED,
    to_version="1.0.0",
    description="New function added for data processing"
)

removed_change = APIChange(
    element=APIElement("old_function", APIType.FUNCTION, "mypackage.deprecated"),
    change_type=ChangeType.REMOVED,
    from_version="1.1.0",
    to_version="2.0.0",
    description="Function removed in major version update"
)

modified_change = APIChange(
    element=method_api,
    change_type=ChangeType.MODIFIED,
    from_version="1.0.0",
    to_version="1.1.0",
    old_signature="transform(self, data)",
    new_signature="transform(self, data: Any) -> Any",
    description="Added type hints to method signature"
)

deprecated_change = APIChange(
    element=APIElement("legacy_method", APIType.METHOD, "mypackage.core"),
    change_type=ChangeType.DEPRECATED,
    from_version="1.1.0",
    to_version="2.0.0",
    description="Method deprecated, use new_method instead"
)

changes = [added_change, removed_change, modified_change, deprecated_change]

print("API Changes:")
print("=" * 60)
for change in changes:
    icon = {"added": "➕", "removed": "➖", "modified": "✏️", "deprecated": "⚠️"}
    print(f"{icon[change.change_type.value]} {change.change_type.value.upper()}: {change.element.full_name}")
    print(f"   Version: {change.from_version or 'N/A'} → {change.to_version}")
    print(f"   Description: {change.description}")
    if change.old_signature and change.new_signature:
        print(f"   Old: {change.old_signature}")
        print(f"   New: {change.new_signature}")
    print()

API Changes:
➕ ADDED: mypackage.utils.process_data
   Version: N/A → 1.0.0
   Description: New function added for data processing

➖ REMOVED: mypackage.deprecated.old_function
   Version: 1.1.0 → 2.0.0
   Description: Function removed in major version update

✏️ MODIFIED: mypackage.core.transform
   Version: 1.0.0 → 1.1.0
   Description: Added type hints to method signature
   Old: transform(self, data)
   New: transform(self, data: Any) -> Any

⚠️ DEPRECATED: mypackage.core.legacy_method
   Version: 1.1.0 → 2.0.0
   Description: Method deprecated, use new_method instead



## 7. Build AnalysisResult with Sample Data

AnalysisResult is the main container that holds all analysis data for a package:

In [7]:
# Create API elements for each version
v1_apis = [
    function_api,
    class_api,
    APIElement("helper_func", APIType.FUNCTION, "mypackage.utils"),
]

v1_1_apis = [
    function_api,  # Still present
    class_api,     # Still present
    APIElement("helper_func", APIType.FUNCTION, "mypackage.utils"),  # Still present
    method_api,    # Added in v1.1
    APIElement("cli_command", APIType.FUNCTION, "mypackage.cli"),    # New in v1.1
]

v2_apis = [
    function_api,  # Still present
    class_api,     # Still present
    # helper_func removed in v2.0
    method_api,    # Still present but modified
    APIElement("cli_command", APIType.FUNCTION, "mypackage.cli"),    # Still present
    constant_api,  # Added in v2.0
    APIElement("new_processor", APIType.CLASS, "mypackage.processors"),  # New in v2.0
]

# Build the complete analysis result
analysis_result = AnalysisResult(
    package_name="mypackage",
    versions=versions,
    api_elements={
        "1.0.0": v1_apis,
        "1.1.0": v1_1_apis,
        "2.0.0": v2_apis
    },
    changes=changes,
    metadata={
        "analysis_tool": "PyPevol",
        "analyzed_at": datetime.now().isoformat(),
        "total_files_analyzed": 15,
        "analysis_duration_seconds": 42.5
    }
)

print("Analysis Result Created:")
print(f"📦 Package: {analysis_result.package_name}")
print(f"📊 Versions: {len(analysis_result.versions)}")
print(f"🔄 Changes: {len(analysis_result.changes)}")
print(f"📈 Total API Elements Tracked: {sum(len(apis) for apis in analysis_result.api_elements.values())}")

# Show version-by-version API counts
print("\nAPI Elements by Version:")
for version, apis in analysis_result.api_elements.items():
    print(f"  v{version}: {len(apis)} APIs")
    for api in apis:
        print(f"    • {api.type.value}: {api.name}")

Analysis Result Created:
📦 Package: mypackage
📊 Versions: 3
🔄 Changes: 4
📈 Total API Elements Tracked: 14

API Elements by Version:
  v1.0.0: 3 APIs
    • function: process_data
    • class: DataProcessor
    • function: helper_func
  v1.1.0: 5 APIs
    • function: process_data
    • class: DataProcessor
    • function: helper_func
    • method: transform
    • function: cli_command
  v2.0.0: 6 APIs
    • function: process_data
    • class: DataProcessor
    • method: transform
    • function: cli_command
    • constant: DEFAULT_CONFIG
    • class: new_processor


## 8. Working with Analysis Results - Filtering and Querying

Once you have an `AnalysisResult`, you can extract useful information about API evolution. Let's explore the built-in filtering and querying capabilities.

In [8]:
# 1. Get APIs introduced in a specific version
def get_introduced_apis(result, version):
    """Get APIs that were introduced in a specific version."""
    if version not in result.api_elements:
        return []
    
    current_apis = {api.get_signature() for api in result.api_elements[version]}
    
    # Find previous version
    version_list = sorted(result.versions, key=lambda v: v.number)
    current_index = next(i for i, v in enumerate(version_list) if v.number == version)
    
    if current_index == 0:
        # First version - all APIs are "introduced"
        return result.api_elements[version]
    
    prev_version = version_list[current_index - 1].number
    prev_apis = {api.get_signature() for api in result.api_elements[prev_version]}
    
    return [api for api in result.api_elements[version] 
            if api.get_signature() not in prev_apis]

# Example: What was introduced in v1.1.0?
introduced_v1_1 = get_introduced_apis(analysis_result, "1.1.0")
print("🆕 APIs introduced in v1.1.0:")
for api in introduced_v1_1:
    print(f"  • {api.type.value}: {api.name} in {api.module}")

# 2. Filter by API type
functions_only = []
classes_only = []
for version, apis in analysis_result.api_elements.items():
    version_functions = [api for api in apis if api.type == APIType.FUNCTION]
    version_classes = [api for api in apis if api.type == APIType.CLASS]
    functions_only.extend([(version, api) for api in version_functions])
    classes_only.extend([(version, api) for api in version_classes])

print(f"\n🔧 Total functions across all versions: {len(functions_only)}")
print(f"🏗️  Total classes across all versions: {len(classes_only)}")

# 3. Find APIs that were removed
def get_removed_apis(result):
    """Find APIs that existed in earlier versions but not in the latest."""
    versions_sorted = sorted(result.versions, key=lambda v: v.number)
    if len(versions_sorted) < 2:
        return []
    
    latest_version = versions_sorted[-1].number
    latest_apis = {api.get_signature() for api in result.api_elements[latest_version]}
    
    removed = set()
    for version in versions_sorted[:-1]:  # All except latest
        for api in result.api_elements[version.number]:
            if api.get_signature() not in latest_apis:
                removed.add(api.get_signature())
    
    return list(removed)

removed_apis = get_removed_apis(analysis_result)
print(f"\n❌ APIs removed in later versions: {len(removed_apis)}")
for signature in removed_apis:
    print(f"  • {signature}")

# 4. Module-based filtering
print(f"\n📁 APIs by module:")
modules = set()
for apis in analysis_result.api_elements.values():
    modules.update(api.module for api in apis)

for module in sorted(modules):
    count = sum(1 for apis in analysis_result.api_elements.values() 
                for api in apis if api.module == module)
    print(f"  • {module}: {count} API occurrences")

🆕 APIs introduced in v1.1.0:
  • method: transform in mypackage.core
  • function: cli_command in mypackage.cli

🔧 Total functions across all versions: 7
🏗️  Total classes across all versions: 4

❌ APIs removed in later versions: 1
  • mypackage.utils.helper_func:function

📁 APIs by module:
  • mypackage.cli: 2 API occurrences
  • mypackage.config: 1 API occurrences
  • mypackage.core: 5 API occurrences
  • mypackage.processors: 1 API occurrences
  • mypackage.utils: 5 API occurrences


## 9. API Lifecycle Analysis

Understanding how APIs evolve over time is crucial for maintaining compatibility. Let's analyze the lifecycle patterns in our data.

In [9]:
# 1. Calculate API stability metrics
def calculate_api_stability(result):
    """Calculate stability metrics for APIs across versions."""
    all_signatures = set()
    version_counts = {}
    
    # Collect all unique API signatures and count appearances
    for version, apis in result.api_elements.items():
        for api in apis:
            signature = api.get_signature()
            all_signatures.add(signature)
            version_counts[signature] = version_counts.get(signature, 0) + 1
    
    # Calculate stability categories
    total_versions = len(result.versions)
    stable_apis = [sig for sig, count in version_counts.items() if count == total_versions]
    unstable_apis = [sig for sig, count in version_counts.items() if count == 1]
    evolving_apis = [sig for sig, count in version_counts.items() if 1 < count < total_versions]
    
    return {
        'total_unique_apis': len(all_signatures),
        'stable_apis': stable_apis,
        'unstable_apis': unstable_apis,
        'evolving_apis': evolving_apis,
        'stability_score': len(stable_apis) / len(all_signatures) if all_signatures else 0
    }

stability = calculate_api_stability(analysis_result)
print("📊 API Stability Analysis:")
print(f"  Total unique APIs: {stability['total_unique_apis']}")
print(f"  Stable APIs (all versions): {len(stability['stable_apis'])}")
print(f"  Unstable APIs (1 version only): {len(stability['unstable_apis'])}")
print(f"  Evolving APIs (some versions): {len(stability['evolving_apis'])}")
print(f"  Stability Score: {stability['stability_score']:.2%}")

# 2. Analyze change patterns
def analyze_change_patterns(result):
    """Analyze patterns in API changes."""
    patterns = {
        'additions': [],
        'removals': [],
        'modifications': [],
        'deprecations': []
    }
    
    for change in result.changes:
        if change.change_type == 'added':
            patterns['additions'].append(change)
        elif change.change_type == 'removed':
            patterns['removals'].append(change)
        elif change.change_type == 'modified':
            patterns['modifications'].append(change)
        elif change.change_type == 'deprecated':
            patterns['deprecations'].append(change)
    
    return patterns

patterns = analyze_change_patterns(analysis_result)
print(f"\n🔄 Change Patterns:")
for pattern_type, changes in patterns.items():
    print(f"  {pattern_type.title()}: {len(changes)}")
    for change in changes[:3]:  # Show first 3 examples
        print(f"    • {change.api_name} in v{change.version}")

# 3. Version-to-version change analysis
def version_to_version_changes(result):
    """Analyze changes between consecutive versions."""
    versions_sorted = sorted(result.versions, key=lambda v: v.number)
    changes_by_transition = {}
    
    for i in range(1, len(versions_sorted)):
        prev_version = versions_sorted[i-1]
        curr_version = versions_sorted[i]
        
        transition = f"{prev_version.number} → {curr_version.number}"
        
        # Find changes in this transition
        transition_changes = [
            change for change in result.changes 
            if change.version == curr_version.number
        ]
        
        changes_by_transition[transition] = transition_changes
    
    return changes_by_transition

version_changes = version_to_version_changes(analysis_result)
print(f"\n📈 Version-to-Version Changes:")
for transition, changes in version_changes.items():
    print(f"  {transition}: {len(changes)} changes")
    
    change_types = {}
    for change in changes:
        change_types[change.change_type] = change_types.get(change.change_type, 0) + 1
    
    for change_type, count in change_types.items():
        print(f"    - {change_type}: {count}")

# 4. Identify breaking changes
breaking_changes = [
    change for change in analysis_result.changes 
    if change.change_type in ['removed', 'modified'] and not change.is_backwards_compatible
]

print(f"\n⚠️  Breaking Changes: {len(breaking_changes)}")
for change in breaking_changes:
    print(f"  • v{change.version}: {change.api_name} ({change.change_type})")
    if change.description:
        print(f"    └─ {change.description}")

📊 API Stability Analysis:
  Total unique APIs: 7
  Stable APIs (all versions): 2
  Unstable APIs (1 version only): 2
  Evolving APIs (some versions): 3
  Stability Score: 28.57%

🔄 Change Patterns:
  Additions: 0
  Removals: 0
  Modifications: 0
  Deprecations: 0

📈 Version-to-Version Changes:
  1.0.0 → 1.1.0: 1 changes
    - ChangeType.MODIFIED: 1
  1.1.0 → 2.0.0: 2 changes
    - ChangeType.REMOVED: 1
    - ChangeType.DEPRECATED: 1

⚠️  Breaking Changes: 0


## 10. Generating Summaries and Statistics

The `AnalysisResult` class provides built-in methods for generating comprehensive summaries and statistics about API evolution.

In [10]:
# 1. Generate a comprehensive summary
summary = analysis_result.get_summary()
print("📋 Analysis Summary:")
print(f"  Package: {summary['package_name']}")
print(f"  Versions analyzed: {summary['total_versions']}")
print(f"  Analysis period: {summary['version_range']['first']} → {summary['version_range']['last']}")
print(f"  Total API changes: {summary['total_changes']}")
print(f"  Unique APIs tracked: {summary['unique_apis']}")

print(f"\n📊 Change Statistics:")
for change_type, count in summary['change_types'].items():
    print(f"  {change_type.title()}: {count}")

print(f"\n🏗️  API Type Distribution:")
for api_type, count in summary['api_types'].items():
    print(f"  {api_type.title()}: {count}")

# 2. Version-specific statistics
print(f"\n📈 Version Details:")
for version_info in summary['versions']:
    print(f"  v{version_info['version']}:")
    print(f"    Released: {version_info['release_date']}")
    print(f"    APIs: {version_info['api_count']}")
    print(f"    Changes: {version_info['changes_count']}")
    
    if version_info['changes_count'] > 0:
        print(f"    Breaking changes: {len([c for c in analysis_result.changes if c.version == version_info['version'] and not c.is_backwards_compatible])}")

# 3. Generate detailed statistics
def generate_detailed_stats(result):
    """Generate detailed statistics about the analysis."""
    stats = {}
    
    # API churn rate (APIs added + removed / total APIs)
    total_changes = len([c for c in result.changes if c.change_type in ['added', 'removed']])
    total_unique_apis = len(set(api.get_signature() for apis in result.api_elements.values() for api in apis))
    stats['churn_rate'] = total_changes / total_unique_apis if total_unique_apis > 0 else 0
    
    # Most active modules (by change count)
    module_changes = {}
    for change in result.changes:
        module = change.api_name.split('.')[0] if '.' in change.api_name else 'root'
        module_changes[module] = module_changes.get(module, 0) + 1
    stats['most_active_modules'] = sorted(module_changes.items(), key=lambda x: x[1], reverse=True)[:5]
    
    # Version with most changes
    version_change_counts = {}
    for change in result.changes:
        version_change_counts[change.version] = version_change_counts.get(change.version, 0) + 1
    stats['most_active_version'] = max(version_change_counts.items(), key=lambda x: x[1]) if version_change_counts else None
    
    # API type evolution
    stats['api_type_trends'] = {}
    for version, apis in result.api_elements.items():
        type_counts = {}
        for api in apis:
            type_counts[api.type.value] = type_counts.get(api.type.value, 0) + 1
        stats['api_type_trends'][version] = type_counts
    
    return stats

detailed_stats = generate_detailed_stats(analysis_result)
print(f"\n📊 Detailed Statistics:")
print(f"API Churn Rate: {detailed_stats['churn_rate']:.2%}")

if detailed_stats['most_active_version']:
    version, change_count = detailed_stats['most_active_version']
    print(f"Most Active Version: v{version} ({change_count} changes)")

print(f"\nMost Active Modules:")
for module, changes in detailed_stats['most_active_modules']:
    print(f"  {module}: {changes} changes")

print(f"\nAPI Type Evolution:")
for version, type_counts in detailed_stats['api_type_trends'].items():
    print(f"  v{version}: {dict(type_counts)}")

# 4. Export summary as JSON for further analysis
summary_json = analysis_result.to_json()
print(f"\n💾 JSON Export Preview (first 200 chars):")
print(summary_json[:200] + "..." if len(summary_json) > 200 else summary_json)

📋 Analysis Summary:
  Package: mypackage
  Versions analyzed: 3
  Analysis period: 1.0.0 → 2.0.0
  Total API changes: 4
  Unique APIs tracked: 7

📊 Change Statistics:
  Added: 1
  Removed: 1
  Modified: 1
  Deprecated: 1

🏗️  API Type Distribution:
  Function: 7
  Class: 4
  Method: 2
  Property: 0
  Constant: 1
  Module: 0

📈 Version Details:
  v1.0.0:
    Released: 2023-01-15
    APIs: 3
    Changes: 1
    Breaking changes: 0
  v1.1.0:
    Released: 2023-03-20
    APIs: 5
    Changes: 1
    Breaking changes: 0
  v2.0.0:
    Released: 2023-06-10
    APIs: 6
    Changes: 2
    Breaking changes: 0

📊 Detailed Statistics:
API Churn Rate: 0.00%
Most Active Version: v2.0.0 (2 changes)

Most Active Modules:
  mypackage: 4 changes

API Type Evolution:
  v1.0.0: {'function': 2, 'class': 1}
  v1.1.0: {'function': 3, 'class': 1, 'method': 1}
  v2.0.0: {'function': 2, 'class': 2, 'method': 1, 'constant': 1}

💾 JSON Export Preview (first 200 chars):
{
  "package_name": "mypackage",
  "versions": 

## 11. Serialization and Persistence

PyPevol provides robust serialization capabilities to save and load analysis results, enabling data persistence and sharing.

In [11]:
# 1. Export to JSON
json_data = analysis_result.to_json()
print("📤 JSON Export:")
print(f"Size: {len(json_data)} characters")

# Parse the JSON to verify structure
import json
parsed_data = json.loads(json_data)
print(f"JSON Structure:")
print(f"  Top-level keys: {list(parsed_data.keys())}")
print(f"  Versions: {len(parsed_data.get('versions', []))}")
print(f"  API elements: {len(parsed_data.get('api_elements', {}))}")
print(f"  Changes: {len(parsed_data.get('changes', []))}")

# 2. Import from JSON (demonstrate round-trip)
print(f"\n📥 JSON Import:")
try:
    restored_result = AnalysisResult.from_json(json_data)
    print(f"✅ Successfully restored analysis result")
    print(f"  Package: {restored_result.package_name}")
    print(f"  Versions: {len(restored_result.versions)}")
    print(f"  API elements: {sum(len(apis) for apis in restored_result.api_elements.values())}")
    print(f"  Changes: {len(restored_result.changes)}")
    
    # Verify data integrity
    original_summary = analysis_result.get_summary()
    restored_summary = restored_result.get_summary()
    
    integrity_check = (
        original_summary['total_versions'] == restored_summary['total_versions'] and
        original_summary['total_changes'] == restored_summary['total_changes'] and
        original_summary['unique_apis'] == restored_summary['unique_apis']
    )
    
    print(f"  Data integrity: {'✅ Passed' if integrity_check else '❌ Failed'}")
    
except Exception as e:
    print(f"❌ Import failed: {e}")

# 3. Save to file and load from file
import tempfile
import os

print(f"\n💾 File Persistence:")
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    f.write(json_data)
    temp_file = f.name

print(f"Saved to: {temp_file}")
print(f"File size: {os.path.getsize(temp_file)} bytes")

# Load from file
try:
    with open(temp_file, 'r') as f:
        file_data = f.read()
    
    file_result = AnalysisResult.from_json(file_data)
    print(f"✅ Successfully loaded from file")
    print(f"  Loaded package: {file_result.package_name}")
    
    # Clean up
    os.unlink(temp_file)
    print(f"🗑️  Temporary file cleaned up")
    
except Exception as e:
    print(f"❌ File operation failed: {e}")

# 4. Create a minimal serializable example
print(f"\n🔬 Minimal Example:")

# Note: VersionInfo now handles both datetime objects and string dates automatically
minimal_result = AnalysisResult(
    package_name="example",
    versions=[VersionInfo("1.0.0", "2023-01-01")],  # String date will be auto-converted
    api_elements={"1.0.0": [APIElement("test_func", APIType.FUNCTION, "example")]},
    changes=[],
    metadata={"source": "manual"}
)

minimal_json = minimal_result.to_json()
print(f"Minimal JSON size: {len(minimal_json)} characters")
print(f"Minimal JSON content (first 500 chars):")
minimal_parsed = json.loads(minimal_json)
print(json.dumps(minimal_parsed, indent=2)[:500] + "..." if len(minimal_json) > 500 else json.dumps(minimal_parsed, indent=2))

# Demonstrate that minimal example can be restored
minimal_restored = AnalysisResult.from_json(minimal_json)
print(f"\n✅ Minimal example round-trip successful: {minimal_restored.package_name}")

# Show that dates are properly handled
if minimal_restored.versions:
    restored_version = minimal_restored.versions[0]
    print(f"   Version: {restored_version.version}")
    print(f"   Date type: {type(restored_version.release_date)}")
    print(f"   Date value: {restored_version.release_date}")

print(f"\n💡 Tip: VersionInfo now automatically converts string dates to datetime objects!")
print(f"   You can pass either datetime objects or date strings like '2023-01-01'.")

📤 JSON Export:
Size: 13099 characters
JSON Structure:
  Top-level keys: ['package_name', 'versions', 'api_elements', 'changes', 'analysis_date', 'metadata', 'summary']
  Versions: 3
  API elements: 3
  Changes: 4

📥 JSON Import:
✅ Successfully restored analysis result
  Package: mypackage
  Versions: 3
  API elements: 14
  Changes: 4
  Data integrity: ✅ Passed

💾 File Persistence:
Saved to: /tmp/tmpy8dfu1z6.json
File size: 13099 bytes
✅ Successfully loaded from file
  Loaded package: mypackage
🗑️  Temporary file cleaned up

🔬 Minimal Example:
Minimal JSON size: 1500 characters
Minimal JSON content (first 500 chars):
{
  "package_name": "example",
  "versions": [
    {
      "version": "1.0.0",
      "release_date": "2023-01-01T00:00:00",
      "python_requires": null,
      "dependencies": [],
      "wheel_url": null,
      "source_url": null,
      "yanked": false,
      "yanked_reason": null,
      "metadata": {}
    }
  ],
  "api_elements": {
    "1.0.0": [
      {
        "name": "

## 12. Summary and Next Steps

🎉 **Congratulations!** You've learned the core APIs of PyPevol. Here's what we covered:

### Core Concepts Mastered:
- ✅ **APIElement**: Representing individual APIs with types, names, and modules  
- ✅ **APIType**: Categorizing different kinds of APIs (functions, classes, methods, etc.)
- ✅ **VersionInfo**: Tracking package versions and release information
- ✅ **APIChange**: Recording how APIs evolve between versions
- ✅ **AnalysisResult**: Comprehensive container for complete evolution analysis

### Key Skills Acquired:
- 🔍 **API Detection**: Identifying private APIs and understanding naming conventions
- 📊 **Data Analysis**: Filtering, querying, and extracting insights from evolution data
- 📈 **Lifecycle Tracking**: Understanding API stability, churn, and breaking changes
- 💾 **Data Persistence**: Saving and loading analysis results for sharing and future use

### What's Next?

Ready to dive deeper? Check out these additional notebooks:

1. **`02_real_package_analysis.ipynb`** - Analyze real PyPI packages like Flask, requests, or click
2. **`03_visualization_examples.ipynb`** - Create stunning charts and graphs of API evolution  
3. **`04_advanced_features.ipynb`** - Explore advanced filtering, comparison, and reporting features
4. **`05_cli_usage.ipynb`** - Master the command-line interface for batch processing

### Real-World Applications:
- 🔧 **Dependency Management**: Track breaking changes in your dependencies
- 📚 **Documentation**: Generate API evolution reports for your users  
- 🔬 **Research**: Study patterns in open-source software evolution
- 🏗️ **Architecture**: Make informed decisions about API design and versioning

### Getting Help:
- 📖 Check the full documentation in the `docs/` folder
- 💡 Browse examples in the `examples/` directory  
- 🐛 Report issues or request features on GitHub
- 💬 Join the community discussions for tips and tricks

Happy analyzing! 🚀